## 初始下载

In [ ]:
%cd /content/
!rm -rf /content/*
!rm -rf /content/SDVAR
!git clone --depth=1 https://github.com/lijrjyan/SDVAR.git
%cd SDVAR
!git log -1
!pwd

In [ ]:
!pwd

# SDVAR

In [ ]:
################## 1. Download checkpoints and build models
import os
import os.path as osp
import torch, torchvision
import random
import numpy as np
import PIL.Image as PImage, PIL.ImageDraw as PImageDraw
# 1. 添加根目录
import sys
sys.path.append('/content/SDVAR')

# 2. 强制刷新模块
import importlib
import models
importlib.reload(models)
setattr(torch.nn.Linear, 'reset_parameters', lambda self: None)     # disable default parameter init for faster speed
setattr(torch.nn.LayerNorm, 'reset_parameters', lambda self: None)  # disable default parameter init for faster speed
from models import VQVAE, build_vae_var_speculative_decoding, build_vae_var
from IPython.display import display

In [ ]:
DRAFT_MODEL_DEPTH = 16    # TODO: =====> please specify MODEL_DEPTH <=====
assert DRAFT_MODEL_DEPTH in {16, 20, 24, 30}
TARGET_MODEL_DEPTH = 30    # TODO: =====> please specify MODEL_DEPTH <=====
assert TARGET_MODEL_DEPTH in {16, 20, 24, 30}

# download checkpoint
hf_home = 'https://huggingface.co/FoundationVision/var/resolve/main'
vae_ckpt, target_var_ckpt, draft_var_ckpt = 'vae_ch160v4096z32.pth', f'var_d{TARGET_MODEL_DEPTH}.pth', f'var_d{DRAFT_MODEL_DEPTH}.pth'
if not osp.exists(vae_ckpt): os.system(f'wget {hf_home}/{vae_ckpt}')
if not osp.exists(target_var_ckpt): os.system(f'wget {hf_home}/{target_var_ckpt}')
if not osp.exists(draft_var_ckpt): os.system(f'wget {hf_home}/{draft_var_ckpt}')

In [ ]:
# set args
seed = 0 #@param {type:"numbe

num_sampling_steps = 200 #@param {type:"slider", min:0, max:1000, step:1}
cfg = 3 #@param {type:"slider", min:1, max:10, step:0.1}
class_labels = (980, 980, 437, 437, 22, 22, 562, 562)  #@param {type:"raw"}
more_smooth = False # True for more smooth output
entry_num = 0 #@param {type:"slider", min:0, max:10, step:1}
sd_mask = 0 #@param {type:"slider", min:0, max:5, step:1}

# seed
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# run faster
tf32 = True
torch.backends.cudnn.allow_tf32 = bool(tf32)
torch.backends.cuda.matmul.allow_tf32 = bool(tf32)
torch.set_float32_matmul_precision('high' if tf32 else 'highest')


In [ ]:
# build vae, var
patch_nums = (1, 2, 3, 4, 5, 6, 8, 10, 13, 16)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# sd_var =. sd需要写一个初始化函数，初始化函数内容包括draft和target model即可

if 'vae' not in globals() or 'draft_var' not in globals() or 'target_var' not in globals():
    vae, draft_var, target_var, sd_var = build_vae_var_speculative_decoding(
        V=4096, Cvae=32, ch=160, share_quant_resi=4,    # hard-coded VQVAE hyperparameters
        device=device, patch_nums=patch_nums,
        num_classes=1000, depth_draft=DRAFT_MODEL_DEPTH, depth_target=TARGET_MODEL_DEPTH, shared_aln=False,
    )

# load checkpoints
vae.load_state_dict(torch.load(vae_ckpt, map_location='cpu'), strict=True)
# draft_var.load_state_dict(torch.load(draft_var_ckpt, map_location='cpu'), strict=True)
# target_var.load_state_dict(torch.load(target_var_ckpt, map_location='cpu'), strict=True)
target_var.load_state_dict(torch.load(target_var_ckpt, map_location='cpu'), strict=True)
draft_var.load_state_dict(torch.load(draft_var_ckpt, map_location='cpu'), strict=True)
vae.eval(), draft_var.eval(), target_var.eval()
for p in vae.parameters(): p.requires_grad_(False)
for p in draft_var.parameters(): p.requires_grad_(False)
for p in target_var.parameters(): p.requires_grad_(False)
print(f'prepare finished.')

In [ ]:
############################# 3. Speculative decoding
import time

for entry_num in range(0, 11):
    print(f"Generating for entry_num = {entry_num}")
    start_time = time.time() # Record the start time
    B = len(class_labels)
    label_B: torch.LongTensor = torch.tensor(class_labels, device=device)
    recon_B3HW = sd_var.sdvar_autoregressive_infer_cfg_sd_test3(
        B=B,
        label_B=label_B,
        cfg=cfg,
        top_k=900,
        top_p=0.95,
        g_seed=seed,
        more_smooth=more_smooth,
        entry_num=entry_num,
        sd_mask=5
    )
    end_time = time.time() # Record the end time
    time_taken = end_time - start_time # Calculate the duration
    print(f"Time taken for entry_num {entry_num}: {time_taken:.2f} seconds") # Print the time taken

    chw = torchvision.utils.make_grid(recon_B3HW, nrow=8, padding=0, pad_value=1.0)
    chw = chw.permute(1, 2, 0).mul_(255).cpu().numpy()
    chw = PImage.fromarray(chw.astype(np.uint8))
    display(chw)

## 常规

In [ ]:
############################# 3. Speculative decoding


for entry_num in range(0, 11):
    print(f"Generating for entry_num = {entry_num}")

    B = len(class_labels)
    label_B: torch.LongTensor = torch.tensor(class_labels, device=device)
    recon_B3HW = sd_var.sdvar_autoregressive_infer_cfg_sd_test3_with_debug_log(
        B=B,
        label_B=label_B,
        cfg=0,
        top_k=900,
        top_p=0.95,
        g_seed=seed,
        more_smooth=more_smooth,
        entry_num=entry_num,
        sd_mask=5
    )

    chw = torchvision.utils.make_grid(recon_B3HW, nrow=8, padding=0, pad_value=1.0)
    chw = chw.permute(1, 2, 0).mul_(255).cpu().numpy()
    chw = PImage.fromarray(chw.astype(np.uint8))
    # display(chw)

下载模型

In [ ]:
  ################## 1. Download checkpoints and build models
import os
import os.path as osp
import torch, torchvision
import random
import numpy as np
import PIL.Image as PImage, PIL.ImageDraw as PImageDraw
from IPython.display import display
setattr(torch.nn.Linear, 'reset_parameters', lambda self: None)     # disable default parameter init for faster speed
setattr(torch.nn.LayerNorm, 'reset_parameters', lambda self: None)  # disable default parameter init for faster speed
from models import VQVAE, build_vae_var_speculative_decoding, build_vae_var
from IPython.display import display

In [ ]:
MODEL_DEPTH = 16    # TODO: =====> please specify MODEL_DEPTH <=====
assert MODEL_DEPTH in {16, 20, 24, 30}


# download checkpoint
hf_home = 'https://huggingface.co/FoundationVision/var/resolve/main'
vae_ckpt, var_ckpt = 'vae_ch160v4096z32.pth', f'var_d{MODEL_DEPTH}.pth'
if not osp.exists(vae_ckpt): os.system(f'wget {hf_home}/{vae_ckpt}')
if not osp.exists(var_ckpt): os.system(f'wget {hf_home}/{var_ckpt}')

# build vae, var
patch_nums = (1, 2, 3, 4, 5, 6, 8, 10, 13, 16)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if 'vae' not in globals() or 'var' not in globals():
    vae, var = build_vae_var(
        V=4096, Cvae=32, ch=160, share_quant_resi=4,    # hard-coded VQVAE hyperparameters
        device=device, patch_nums=patch_nums,
        num_classes=1000, depth=MODEL_DEPTH, shared_aln=False,
    )

# load checkpoints
vae.load_state_dict(torch.load(vae_ckpt, map_location='cpu'), strict=True)
var.load_state_dict(torch.load(var_ckpt, map_location='cpu'), strict=True)
vae.eval(), var.eval()
for p in vae.parameters(): p.requires_grad_(False)
for p in var.parameters(): p.requires_grad_(False)
print(f'prepare finished.')

In [ ]:
############################# 2. Sample with classifier-free guidance

# set args
seed = 0 #@param {type:"number"}
torch.manual_seed(seed)
num_sampling_steps = 250 #@param {type:"slider", min:0, max:1000, step:1}
cfg = 4 #@param {type:"slider", min:1, max:10, step:0.1}
class_labels = (980, 980, 437, 437, 22, 22, 562, 562)  #@param {type:"raw"}
more_smooth = False # True for more smooth output

# seed
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# run faster
tf32 = True
torch.backends.cudnn.allow_tf32 = bool(tf32)
torch.backends.cuda.matmul.allow_tf32 = bool(tf32)
torch.set_float32_matmul_precision('high' if tf32 else 'highest')

# sample
B = len(class_labels)
label_B: torch.LongTensor = torch.tensor(class_labels, device=device)
# with torch.inference_mode():
#     with torch.autocast('cuda', enabled=True, dtype=torch.float16, cache_enabled=True):    # using bfloat16 can be faster
recon_B3HW = var.autoregressive_infer_cfg(B=B, label_B=label_B, cfg=cfg, top_k=900, top_p=0.95, g_seed=seed, more_smooth=more_smooth)

chw = torchvision.utils.make_grid(recon_B3HW, nrow=8, padding=0, pad_value=1.0)
chw = chw.permute(1, 2, 0).mul_(255).cpu().numpy()
chw = PImage.fromarray(chw.astype(np.uint8))
display(chw)

In [ ]:
############################# 2. Sample with classifier-free guidance

# set args
seed = 0 #@param {type:"number"}
torch.manual_seed(seed)
num_sampling_steps = 250 #@param {type:"slider", min:0, max:1000, step:1}
cfg = 4 #@param {type:"slider", min:1, max:10, step:0.1}
class_labels = (980, 980, 437, 437, 22, 22, 562, 562)  #@param {type:"raw"}
more_smooth = False # True for more smooth output

# seed
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# run faster
tf32 = True
torch.backends.cudnn.allow_tf32 = bool(tf32)
torch.backends.cuda.matmul.allow_tf32 = bool(tf32)
torch.set_float32_matmul_precision('high' if tf32 else 'highest')

# sample
B = len(class_labels)
label_B: torch.LongTensor = torch.tensor(class_labels, device=device)
# with torch.inference_mode():
#     with torch.autocast('cuda', enabled=True, dtype=torch.float16, cache_enabled=True):    # using bfloat16 can be faster
recon_B3HW = var.autoregressive_infer_cfg1(B=B, label_B=label_B, cfg=cfg, top_k=900, top_p=0.95, g_seed=seed, more_smooth=more_smooth)

chw = torchvision.utils.make_grid(recon_B3HW, nrow=8, padding=0, pad_value=1.0)
chw = chw.permute(1, 2, 0).mul_(255).cpu().numpy()
chw = PImage.fromarray(chw.astype(np.uint8))
display(chw)